In [118]:
import pandas as pd
import torch
import re
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, balanced_accuracy_score

In [119]:
results_cls = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/results_cls_two_models_val.csv')
#results_cls.to_csv('/data/kpusteln/Fetal-RL/test_data/results_cls2.csv', index = False)

In [120]:
results_cls

,index,video,predict,gt,ps
0,2_1_1,2_1,1,1,0.107914
1,2_1_2,2_1,1,1,0.107914
2,2_1_3,2_1,1,1,0.107914
3,2_1_4,2_1,1,1,0.107914
4,2_1_5,2_1,1,1,0.107914
...,...,...,...,...,...
35124,692_3_160,692_3,0,0,0.229240
35125,692_3_161,692_3,0,0,0.229240
35126,692_3_162,692_3,0,0,0.229240
35127,692_3_163,692_3,5,5,0.229240


In [121]:
result_filtered = results_cls[(results_cls['gt'] == 2) | (results_cls['gt'] == 4) | (results_cls['gt'] == 6)]

In [122]:
result_filtered

,index,video,predict,gt,ps
72,2_1_73,2_1,2,2,0.107914
73,2_1_74,2_1,2,2,0.107914
74,2_1_75,2_1,2,2,0.107914
75,2_1_76,2_1,2,2,0.107914
76,2_1_77,2_1,2,2,0.107914
...,...,...,...,...,...
34960,680_3_39,680_3,6,6,0.167642
34961,680_3_40,680_3,5,6,0.167642
34962,680_3_41,680_3,5,6,0.167642
34963,680_3_42,680_3,5,6,0.167642


In [123]:
results_new = pd.DataFrame()

In [124]:
results_new['index'] = result_filtered['index']
results_new['video'] = result_filtered['video']
results_new['ps'] = result_filtered['ps']
results_new['predict'] = result_filtered['predict']

In [125]:
results_new

,index,video,ps,predict
72,2_1_73,2_1,0.107914,2
73,2_1_74,2_1,0.107914,2
74,2_1_75,2_1,0.107914,2
75,2_1_76,2_1,0.107914,2
76,2_1_77,2_1,0.107914,2
...,...,...,...,...
34960,680_3_39,680_3,0.167642,6
34961,680_3_40,680_3,0.167642,5
34962,680_3_41,680_3,0.167642,5
34963,680_3_42,680_3,0.167642,5


In [117]:
results_new.to_csv('/data/kpusteln/Fetal-RL/test_data/results_two_models_cls_filtered_val_gt.csv', index = False)

In [2]:
results_filtered = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/results_cls_filtered.csv')

In [3]:
test_data = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/test_data_full.csv')

In [10]:
merged = pd.merge(test_data, results_filtered, on='index', how='right')

In [16]:
merged.drop('video_y', axis=1, inplace=True)

In [18]:
merged.drop('result', axis=1, inplace=True)

In [24]:
merged.rename(columns={'video_x': 'video'}, inplace=True)

In [26]:
merged.to_csv('/data/kpusteln/Fetal-RL/test_data/test_data_filtered.csv', index = False)

In [2]:
results_reg = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/results_reg_two_models_test.csv')

In [3]:
results_reg

,index,video,measures
0,1_1_13,1_1,17.912975
1,1_1_23,1_1,18.377446
2,1_1_24,1_1,18.385447
3,1_1_25,1_1,18.410865
4,1_1_26,1_1,18.422236
...,...,...,...
4236,688_3_49,688_3,4.941322
4237,688_3_50,688_3,4.926683
4238,688_3_51,688_3,4.943198
4239,699_3_176,699_3,5.098561


In [4]:
gt_data = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_test.csv')

In [9]:
gt_data_new = pd.DataFrame()
gt_data_new['index'] = gt_data['index']
gt_data_new['gt_measure'] = gt_data['measure']

In [13]:
data_merged = pd.merge(results_reg, gt_data_new, on='index', how='left')

In [39]:
data_merged

,index,video,measures,gt_measure
0,1_1_13,1_1,17.912975,20.19
1,1_1_23,1_1,18.377446,20.19
2,1_1_24,1_1,18.385447,20.19
3,1_1_25,1_1,18.410865,20.19
4,1_1_26,1_1,18.422236,20.19
...,...,...,...,...
4236,688_3_49,688_3,4.941322,5.13
4237,688_3_50,688_3,4.926683,5.13
4238,688_3_51,688_3,4.943198,5.13
4239,699_3_176,699_3,5.098561,5.15


In [25]:
mean_data = data_merged.groupby('video', as_index=False).mean()

In [36]:
mean_absolute_error(mean_data['gt_measure'], mean_data['measures'])

0.6843471803129886

In [54]:
print('MAE', mean_absolute_percentage_error(mean_data['gt_measure'], mean_data['measures']))
print('RMSE', mean_squared_error(mean_data['gt_measure'], mean_data['measures'], squared=False))
print('MAPE', mean_absolute_error(mean_data['gt_measure'], mean_data['measures']))

MAE 0.06327097406649193
RMSE 2.520036428206987
MAPE 0.6843471803129886


In [55]:
print('MAE', mean_absolute_percentage_error(data_merged['gt_measure'], data_merged['measures']))
print('RMSE', mean_squared_error(data_merged['gt_measure'], data_merged['measures'], squared=False))
print('MAPE', mean_absolute_error(data_merged['gt_measure'], data_merged['measures']))

MAE 0.04434770855150079
RMSE 1.5517260471436354
MAPE 0.362762138459599


In [ ]:
"1 - head"
"2 - abdomen"
"3 - femur"

In [81]:
data_merged['type'] = data_merged['video'].str.split('_', expand=True).iloc[:, 1]

In [89]:
data_merged_grouped = data_merged.groupby('video', as_index=False).mean()

In [91]:
data_merged_grouped['type'] = data_merged_grouped['video'].str.split('_', expand=True).iloc[:, 1]

In [98]:
data_femur = data_merged_grouped[data_merged_grouped['type'] == '3']
data_head = data_merged_grouped[data_merged_grouped['type'] == '1']
data_abdomen = data_merged_grouped[data_merged_grouped['type'] == '2']

In [105]:
data_femur

,video,measures,gt_measure,type
0,112_3,3.028816,3.37,3
4,126_3,2.986336,2.95,3
5,130_3,3.093793,2.75,3
6,135_3,4.222304,4.44,3
7,137_3,6.187977,6.13,3
...,...,...,...,...
77,74_3,4.993599,5.19,3
79,81_3,3.769137,3.71,3
80,86_3,5.185333,5.23,3
81,88_3,7.117405,7.48,3


In [106]:
print('MAE', mean_absolute_percentage_error(data_femur['gt_measure'], data_femur['measures']))
print('RMSE', mean_squared_error(data_femur['gt_measure'], data_femur['measures'], squared=False))
print('MAPE', mean_absolute_error(data_femur['gt_measure'], data_femur['measures']))

MAE 0.04635212711442501
RMSE 0.26321143194636654
MAPE 0.20061725665072538


In [102]:
data_abdomen

,video,measures,gt_measure,type
3,126_2,14.468736,12.86,2
31,29_2,20.398741,20.82,2
42,39_2,26.930304,28.45,2
48,484_2,4.111499,19.17,2
51,495_2,17.448423,17.39,2
55,535_2,18.041521,17.59,2
71,674_2,4.887397,21.56,2
73,68_2,11.769169,11.68,2
74,692_2,20.120240,21.25,2


In [107]:
print('MAE', mean_absolute_percentage_error(data_abdomen['gt_measure'], data_abdomen['measures']))
print('RMSE', mean_squared_error(data_abdomen['gt_measure'], data_abdomen['measures'], squared=False))
print('MAPE', mean_absolute_error(data_abdomen['gt_measure'], data_abdomen['measures']))

MAE 0.20526780994666127
RMSE 7.537319709173787
MAPE 4.112185366857097


In [108]:
data_head

,video,measures,gt_measure,type
1,117_1,19.859516,19.85,1
2,123_1,18.034299,17.98,1
14,1_1,18.351954,20.19,1
28,27_1,14.071474,14.92,1
34,31_1,19.496624,19.68,1
69,64_1,22.051835,24.53,1
78,80_1,24.592137,25.53,1


In [109]:
print('MAE', mean_absolute_percentage_error(data_head['gt_measure'], data_head['measures']))
print('RMSE', mean_squared_error(data_head['gt_measure'], data_head['measures'], squared=False))
print('MAPE', mean_absolute_error(data_head['gt_measure'], data_head['measures']))

MAE 0.042641148761771934
RMSE 1.2624220424704118
MAPE 0.9071130669522264


In [4]:
results_reg.drop('index', axis=1, inplace=True)

In [5]:
results_reg.aggregate(['mean', 'std'], axis=0)

,video,measures
mean,inf,11.326495
std,NaN,8.047107


In [6]:
len(results_reg['video'].unique())

141

In [7]:
results_reg.groupby('video').aggregate(['mean', 'std'], axis=0).to_csv('/data/kpusteln/Fetal-RL/test_data/results_reg_grouped.csv')

In [8]:
data = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/results_reg_two_models_grouped.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/data/kpusteln/Fetal-RL/test_data/results_reg_two_models_grouped.csv'

In [45]:
data

,video,mean,std
0,1000_3,3.778001,0.027507
1,1001_1,13.876002,0.129915
2,1001_2,12.901161,0.476043
3,1001_3,2.272100,0.020860
4,1005_2,15.027100,NaN
...,...,...,...
165,981_2,19.025668,0.163104
166,981_3,4.581327,0.022991
167,986_3,2.337426,0.014451
168,996_2,16.280219,NaN


In [15]:
results = pd.read_csv('/data/kpusteln/Fetal-RL/test_data/results_cls_augm_val.csv')

In [16]:
wynik = results['gt'][0]

In [17]:
wyniki = [int(re.findall("\d+", results['gt'][i])[0]) for i in range(len(results['gt']))]

In [18]:
results['gt'] = wyniki

In [19]:
results

,index,video,predict,gt
0,5_1_1,5_1,1,1
1,5_1_3,5_1,1,1
2,5_1_4,5_1,1,1
3,5_1_5,5_1,1,1
4,5_1_6,5_1,1,1
...,...,...,...,...
13230,703_3_50,703_3,5,5
13231,703_3_51,703_3,5,5
13232,703_3_52,703_3,5,5
13233,703_3_53,703_3,5,5


In [24]:
recall_score(results['gt'], results['predict'], average='micro')

0.8597657725727238

In [27]:
accuracy_score(results['gt'], results['predict'])

0.8597657725727238

In [9]:
"""new_labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - abdomen non-standard plane
4 - abdomen standard plane
5 - femur non standard plane
6 - femur standard plane"""

'new_labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - abdomen non-standard plane\n4 - abdomen standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [15]:
weights = (1 - precision_score(results['gt'], results['predict'], average=None))/(1 - precision_score(results['gt'], results['predict'], average=None)).sum()

In [21]:
weights = torch.tensor(weights, dtype=torch.float32)
torch.save(weights, '/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/precision_weights.pt')

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [22]:
torch.load('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/precision_weights.pt')

tensor([0.1298, 0.1067, 0.0632, 0.0293, 0.4131, 0.1264, 0.1315])

In [11]:
balanced_accuracy_score(results['gt'], results['predict'])

0.6946790114668444

In [57]:
abdomen_train = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_train.csv')

In [2]:
data_train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_train.csv')

In [3]:
data_train['Class'].value_counts()

3    63443
5    33952
6    27206
0    19524
1     9472
4     2965
2     1639
Name: Class, dtype: int64

In [4]:
n_samples = [19524, 9472, 1639, 63443, 2965, 33952, 27206]
normedWeights = [(x / sum(n_samples)) for x in n_samples]
normedWeights = torch.tensor(normedWeights)

In [5]:
normedWeights = 1.0/normedWeights

In [6]:
normedWeights = normedWeights / normedWeights.sum()

In [8]:
torch.save(normedWeights, '/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/normedWeights.pt')

In [9]:
normedWeights = torch.load('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/normedWeights.pt')

In [10]:
normedWeights

tensor([0.0432, 0.0890, 0.5144, 0.0133, 0.2843, 0.0248, 0.0310])